<div class='bar_title'></div>

*Introduction to Data Science (IDS)*

# Assignment 9 - Descriptive Machine Learning Solutions

Gunther Gust / Vanessa Haustein <br>
Chair for Enterprise AI<br>
Data Driven Decisions (D3) Group<br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="https://raw.githubusercontent.com/GuntherGust/tds2_data/main/images/d3.png?raw=1" style="width:20%; float:left;" />

# Exercise 1: Compare models
In this exercise, you will learn how to evaluate and compare machine learning models using AIC and BIC to select the best model based on its descriptive capability.

You will work with the penguins dataset again and train three different models to predict the body mass of a penguin in order to see which one is the most appropriate.

In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# Load Penguins dataset
penguins = fetch_openml(data_id=42585, as_frame=True)
df = penguins.frame
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


(a) Decide whether it makes sense to deal with the missing values or not and modify the data accordingly. Is there anything else that should be removed from the datset?

In [3]:
df.isna().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

First, we notice that not many values are missing and most of them are coming from the binary variable `sex`. To impute this one might make sense (using for example the `most-frequent` method) if we were sure that there is no underlying pattern and if it were a high amount of data. But since removing the instances where `sex` has no value also removes the other missing values, it is valid to just drop all NAs in this case:

In [4]:
df.dropna(subset=['sex']).isna().sum()

species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [5]:
df = df.dropna()

Looking at the unique values of the column `sex`, we see that there are three:

In [6]:
print(df['sex'].unique())
print(len(df[df['sex'] == '_']))

['MALE', 'FEMALE', '_']
Categories (3, object): ['FEMALE', 'MALE', '_']
1


Since there is only one instance of sex = '_', we should remove that one, too.

In [7]:
df = df[df['sex'] != '_']

(b) Define features and target.

In [8]:
X = df.drop(columns=['body_mass_g'])
y = df['body_mass_g']

(c) Next, encode the categroical variables.

In [9]:
ohe_encoder = OneHotEncoder(sparse_output=False)
X_cat_ohe = pd.DataFrame(ohe_encoder.fit_transform(X[['species', 'island', 'sex']]),
                                 index=X.index, columns=ohe_encoder.get_feature_names_out())

In [10]:
X_cat_ohe

,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
338,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
340,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
341,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
342,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [11]:
X_final = pd.concat([X.drop(columns=['species', 'island', 'sex']), X_cat_ohe], axis=1)
X_final

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,39.1,18.7,181.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,39.5,17.4,186.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,40.3,18.0,195.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,36.7,19.3,193.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,39.3,20.6,190.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
338,47.2,13.7,214.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
340,46.8,14.3,215.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
341,50.4,15.7,222.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
342,45.2,14.8,212.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


(d) Fit three different regression models: one Linear Regression Model with all features, one Polynomial Regression Model of degree 2 and one Linear Regression Model with only the numerical features.

For the polynomial model, you will have to transform your features into polynomial features. Use the function `PolynomialFeatures` for this. It will extend simple linear models to handle non-linear patterns by creating new, higher-degree features from your original data. For more information, check out [this link](https://scikit-learn.org/1.5/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

In [12]:
# Model 1: Linear Regression with all features
model1 = LinearRegression()
model1.fit(X_final, y)

# Model 2: Polynomial Regression (degree 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_final)
model2 = LinearRegression()
model2.fit(X_poly, y)

# Model 3: Linear Regression with numerical features
X_num = X.drop(columns=['species', 'island', 'sex'])
model3 = LinearRegression()
model3.fit(X_num, y)

LinearRegression()

(e) Now, write a function to calculate AIC and BIC in order to evaluate your models. You will have to calculate the metrics via the explicit formula. The function should take the model, the feature dataset and the target dataset as inputs and should return AIC and BIC. Make sure to transform the feature data again to polynomial features when looking at the second model.

In [13]:
def calculate_aic_bic(model, X, y, is_polynomial=False, poly=None):
    # Predictions
    if is_polynomial and poly:
        X = poly.transform(X)
    y_pred = model.predict(X)

    # Residual sum of squares
    rss = np.sum((y - y_pred) ** 2)

    # Number of parameters
    k = len(model.coef_) + 1  # +1 for intercept

    # Number of observations
    n = len(y)

    # AIC and BIC calculations
    aic = n * np.log(rss / n) + 2 * k
    bic = n * np.log(rss / n) + k * np.log(n)

    return aic, bic

In [14]:
# AIC and BIC for Model 1
aic1, bic1 = calculate_aic_bic(model1, X_final, y)

# AIC and BIC for Model 2
aic2, bic2 = calculate_aic_bic(model2, X_final, y, is_polynomial=True, poly=poly)

# AIC and BIC for Model 3
aic3, bic3 = calculate_aic_bic(model3, X_num, y)

# Print results
print(f"Model 1: AIC={aic1}, BIC={bic1}")
print(f"Model 2: AIC={aic2}, BIC={bic2}")
print(f"Model 3: AIC={aic3}, BIC={bic3}")

Model 1: AIC=3786.2290454572408, BIC=3831.926755337006
Model 2: AIC=3876.1001710765227, BIC=4173.135285294998
Model 3: AIC=3982.5407870050144, BIC=3997.7733569649363


(f) What is the difference between AIC and BIC? Which model would you have expected to perform best in this case and why? Do the AIC and BIC values from above confirm this?

BIC aims to identify the model that is most likely to be the true data-generating process.  
It uses a stronger complexity penalty $k \ln(n)$, which grows with the sample size $n$. With growing $n$, $k\ln(n)$ grows faster than $2k$, so the BIC becomes much stricter $\rightarrow$ BIC will almost always choose a simpler model compared to AIC.

The AIC and BIC values alone don't tell you if the model is good or bad, but when comparing them across different models, they can be interpreted. Since lower values indicate a better model fit for both AIC and BIC, the linear regression model with all features should be chosen here. It can be deduced from comparison to the third model, that adding the features `species`, `island` and `sex` improves the accuracy without making it too complex.

Comparing Model 2 and 3, we see that Model 2 has lower AIC but higher BIC than model 3. A selection of one of these two depends on the goal of the analysis:
- If the goal is to avoid overfitting and prioritize simplicity and generalizability, Model 3 (lower BIC) would be the better choice.
- If the goal is purely to fit the data well, and the complexity of the model is not a concern, Model 2 (lower AIC) could be considered. However, caution should be taken about potential overfitting.

# Exercise 2: PCA

In this exercise, you will perform PCA on the breast cancer dataset. The goal is to visualize the type of cancer based on the other features. Additionally, we want to find out how many components are needed to explain most of the variance in the dataset.

In [15]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from lets_plot import *
LetsPlot.setup_html()

In [16]:
breast_cancer = load_breast_cancer()
df_bc = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)
df_bc['cancer_type'] = breast_cancer.target

df_bc['cancer_type'] = df_bc['cancer_type'].apply(lambda x: 'benign' if x == 0 else 'malignant')
df_bc.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,cancer_type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,benign
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,benign
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,benign
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,benign
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,benign


(a) Separate the features from the target in the dataset and standardize them.

In [17]:
features_breast_cancer = df_bc.columns.tolist()
features_breast_cancer = features_breast_cancer[:-1]

x_breast_cancer = df_bc.loc[:, features_breast_cancer].values

x_breast_cancer = StandardScaler().fit_transform(x_breast_cancer)

(b) Perform PCA on the data. We want to keep two components in order to plot it in a 2D plot later. Append the target columnn to the dataframe you create with the PCA that contains the first two pricipal component values and plot the results. Additionally, report which percentage of the total variance can be explained by the first and second PC. How much variance does the dimensionality reduction from all features to only the first two PCs explain?

In [18]:
pca_breast_cancer = PCA(n_components=2)

In [19]:
pc_breast_cancer = pca_breast_cancer.fit_transform(x_breast_cancer)
pc_df_breast_cancer = pd.DataFrame(data=pc_breast_cancer, columns=['pc1', 'pc2'])
final_df_breast_cancer = pd.concat([pc_df_breast_cancer, df_bc[['cancer_type']]], axis=1)
final_df_breast_cancer.head()

,pc1,pc2,cancer_type
0,9.192837,1.948583,benign
1,2.387802,-3.768172,benign
2,5.733896,-1.075174,benign
3,7.122953,10.275589,benign
4,3.935302,-1.948072,benign


In [20]:
(
    ggplot(final_df_breast_cancer, aes(x='pc1', y='pc2', color='cancer_type'))
    + geom_point()
)

In [21]:
variance_ratio = pca_breast_cancer.explained_variance_ratio_

print(f"Principal Component 1 explains {variance_ratio[0] * 100}% of total variance.")
print(f"Principal Component 2 explains {variance_ratio[1] * 100}% of total variance.")
print(f"Dimensionality reducion from {len(df_bc.columns)-1} to {len(variance_ratio)} "
      f"leaves {sum(variance_ratio) * 100}% of total variance.")

Principal Component 1 explains 44.27202560752637% of total variance.
Principal Component 2 explains 18.971182044033085% of total variance.
Dimensionality reducion from 30 to 2 leaves 63.243207651559466% of total variance.


(c) Now we want to see how much more variance can be explained by using up to 10 PCs. Create an informative elbow plot: Save the explained variance ratio of each additional PC in a list and plot it using lets-plot. Interpreting the plot, how many components would be reasonable to be used?

In [22]:
pca_breast_cancer = PCA(n_components=10)

In [23]:
pc_breast_cancer = pca_breast_cancer.fit_transform(x_breast_cancer)
pc_df_breast_cancer = pd.DataFrame(data=pc_breast_cancer)
final_df_breast_cancer = pd.concat([pc_df_breast_cancer, df_bc[['cancer_type']]], axis=1)
final_df_breast_cancer.head()

,0,1,2,3,4,5,6,7,8,9,cancer_type
0,9.192837,1.948583,-1.123166,-3.633731,1.195110,1.411424,2.159370,-0.398407,-0.157118,-0.877402,benign
1,2.387802,-3.768172,-0.529293,-1.118264,-0.621775,0.028656,0.013358,0.240988,-0.711905,1.106995,benign
2,5.733896,-1.075174,-0.551748,-0.912083,0.177086,0.541452,-0.668166,0.097374,0.024066,0.454275,benign
3,7.122953,10.275589,-3.232790,-0.152547,2.960878,3.053422,1.429911,1.059565,-1.405440,-1.116975,benign
4,3.935302,-1.948072,1.389767,-2.940639,-0.546747,-1.226495,-0.936213,0.636376,-0.263805,0.377704,benign


In [24]:
variance_ratio = pca_breast_cancer.explained_variance_ratio_
df = pd.DataFrame({
    'Principal Component': [f'PC{i+1}' for i in range(len(variance_ratio))],
    'Explained Variance Ratio': variance_ratio
})

In [25]:
(
    ggplot(df, aes(x='Principal Component', y='Explained Variance Ratio'))
    + geom_line()
    + geom_point()
)

The curve flattens out after PC4: PC4 adds around 7% of retained variance, whereas PC5 only adds 5%. All remaining components only explain less than 4% each. For dimensionality reduction, the choice of 3 or 4 components would be best to still explain the majority of the variance.

# Exercise 3: Clustering

You are provided with two different clustering sets. Test the performance of two different clustering algorithms: kMeans and DBSCAN. on those datasets. Find out which algorithm works best for which data and why.

In [26]:
from lets_plot import *
LetsPlot.setup_html()
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [27]:
df1 = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/first_clustering_dataset.csv', usecols=['col1', 'col2'])
df2 = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/second_clustering_dataset.csv', usecols=['col1', 'col2'])

(a) First, plot each dataset.

In [28]:
(
    ggplot(pd.DataFrame(df1), aes(x='col1', y='col2'))
    + geom_point()
    + ggtitle('First Dataset')
)

In [29]:
(
    ggplot(pd.DataFrame(df2), aes(x='col1', y='col2'))
    + geom_point()
    + ggtitle('Second Dataset')
)

## KMeans
(b) Now, apply kMeans to the three datasets. Plot the result.

### First Dataset with KMeans

In [30]:
kmeans1 = KMeans(n_clusters=5, max_iter = 1000, random_state = 42)
kmeans1.fit(df1)

KMeans(max_iter=1000, n_clusters=5, random_state=42)

In [31]:
df1_plot = pd.DataFrame(df1)
df1_plot['label'] = kmeans1.labels_

# Plotting using lets-plot
(
    ggplot(df1_plot)
    + geom_point(aes(x='col1', y='col2', color='label'), size=3)
    + ggtitle('K-Means for First Dataset')
            + scale_color_discrete(name='Cluster')

)

### Second Dataset with KMeans

In [32]:
kmeans2 = KMeans(n_clusters=2, max_iter = 1000, random_state = 42)
kmeans2.fit(df2)

KMeans(max_iter=1000, n_clusters=2, random_state=42)

In [33]:
df2_plot = pd.DataFrame(df2)
df2_plot['label'] = kmeans2.labels_

# Plotting using lets-plot
(
    ggplot(df2_plot)
    + geom_point(aes(x='col1', y='col2', color='label'), size=3)
    + ggtitle('K-Means for Second Dataset')
    + scale_color_discrete(name='Cluster')

)

## DBSCAN
(c) Next, use the DBSCAN algorithm:

### First Dataset with DBSCAN

In [34]:
DBS1 = DBSCAN(eps=0.25, min_samples=10).fit(df1)

In [35]:
df1_plot = pd.DataFrame(df1)
df1_plot['label'] = DBS1.labels_

# Plotting using lets-plot
(
    ggplot(df1_plot)
    + geom_point(aes(x='col1', y='col2', color='label'), size=3)
    + ggtitle('DBSCAN for First Dataset')
    + scale_color_discrete(name='Cluster')

)

### Second Dataset with DBSCAN

In [36]:
DBS2 = DBSCAN(eps=0.25, min_samples=10).fit(df2)

In [37]:
df2_plot = pd.DataFrame(df2)
df2_plot['label'] = DBS2.labels_

# Plotting using lets-plot
(
    ggplot(df2_plot)
    + geom_point(aes(x='col1', y='col2', color='label'), size=3)
    + ggtitle('DBSCAN for Second Dataset')
    + scale_color_discrete(name='Cluster')
)

# Exercise 4: Association Rule Mining

You are provided with a retail shopping basket and your task is to determine which association rules hold here.

In [38]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [39]:
retail_shopping_basket = {'ID':[1,2,3,4,5,6],
                         'Basket':[['Beer', 'Diaper', 'Pretzels', 'Chips', 'Aspirin'],
                                   ['Diaper', 'Beer', 'Chips', 'Lotion', 'Juice', 'BabyFood', 'Milk'],
                                   ['Soda', 'Chips', 'Milk'],
                                   ['Soup', 'Beer', 'Diaper', 'Milk', 'IceCream'],
                                   ['Soda', 'Coffee', 'Milk', 'Bread'],
                                   ['Beer', 'Chips']
                                  ]
                         }

(a) First, one-hot encode the basket so that it can be processed by the apriori function.

In [40]:
te = TransactionEncoder()
te_basket = te.fit(retail_shopping_basket['Basket']).transform(retail_shopping_basket['Basket'])
te_basket = pd.DataFrame(te_basket, columns=te.columns_)

(b) Now, use the apriori function to find the most frequent itemsets and then generate association rules for them.

In [41]:
frequent_itemsets = apriori(te_basket, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.666667,(Beer)
1,0.666667,(Chips)
2,0.500000,(Diaper)
3,0.666667,(Milk)
4,0.500000,"(Beer, Chips)"
5,0.500000,"(Beer, Diaper)"


In [42]:
rules = association_rules(df=frequent_itemsets, num_itemsets=2, metric="lift", min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Beer),(Chips),0.666667,0.666667,0.5,0.75,1.125,1.0,0.055556,1.333333,0.333333,0.60,0.25,0.750
1,(Chips),(Beer),0.666667,0.666667,0.5,0.75,1.125,1.0,0.055556,1.333333,0.333333,0.60,0.25,0.750
2,(Beer),(Diaper),0.666667,0.500000,0.5,0.75,1.500,1.0,0.166667,2.000000,1.000000,0.75,0.50,0.875
3,(Diaper),(Beer),0.500000,0.666667,0.5,1.00,1.500,1.0,0.166667,inf,0.666667,0.75,1.00,0.875


(c) Considering the default metric of the `association_rule` function, what is the most frequently bought itemset?

Since the default metric is `confidence`, {Diaper, Beer} is the most associated itemset in the given basket.